# 전처리에 앞서..

보험사로부터 받은 보험 제안서에는 다양한 테이블 데이터가 존재한다.   
데이터를 살펴보고 나에게 꼭 필요한 보험을 가리는 분석을 위해 전처리를 수행하였다.

가장 먼저 수행한 일은 PDF 파일의 Table 데이터를 Excel로 추출하고 이를 Pandas로 불러 올 수 있게 만드는 일이다.

PDF-> excel로 변환 작업은 [ilovepdf.com](https://www.ilovepdf.com/ko)을 통해 수행하였고, 후처리를 통해 DataFrame으로 불러올 수 있는 형태로 만들었다.

데이터 원본의 예시는 아래와 같았다.
<p align="center">
  <img src="https://drive.google.com/uc?id=1i5ALZXRPlTvuIU6GJJnrZ0CuiQ0SeMvW" stype="display: inline-block;" align="center" width="45%">
  <img src="https://drive.google.com/uc?id=1XMEJLv9dJLAZjVDRsJgkdGlC_3_Wizsz" style="display: inline-block;" align="center" width="45%">
</p>


#### 설계 받은 보험 특징
보험 설계사로부터 제안 받은 보험은 크게 두 가지 종류로 나누어 볼 수 있다.  
1. 특정 질병을 크게 보장받는 것이 아니라 다양한 질병을 적당한 선에서 보장 받을 수 있는 것이다.  
2. 병원 입원 시 생활의 불편함을 해소할 수 있는 간병인에 대한 보장이 들어있다.  


이제 데이터를 분석 해보자.
여기서 사용할 데이터는 PDF에서 테이블을 추출하고 엑셀에서 간단한 전처리를 거친 결과다.  
이렇게 앞으로 사용할 Raw Data를 만들었다.

# 전처리 시작

### 모듈 불러오기

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)

In [2]:
data_url = "https://docs.google.com/uc?id=1lYoZnDPHbXD3JAT8lD1eO0w9__0ZkC8U"
data = pd.read_excel(data_url)
data.head()

,유형1,유형2,보험명,상세 설명,보장금액,월 보험료,기본 보험료,할증 보험료,납입기간/보험기간,그룹
0,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원 제외),"상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7305,7305,NaN,20년납 100세만기,NaN
1,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원),"상해사고로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한 간병인을 ...",30000,393,393,NaN,20년납 100세만기,NaN
2,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원 제외),보험기간 중 질병으로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 ...,150000,25297,17265,8032.0,20년납 100세만기,NaN
3,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원),"보험기간 중 질병으로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한...",10000,3438,2374,1064.0,20년납 100세만기,NaN
4,입원,간병인2,"(체증형,20년후2배)상해 입원 간병인 사용일당(1 일이상)(요양병원 제외)","상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7695,7695,NaN,20년납\n100세만기,NaN


## 기본적인 EDA

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   유형1        71 non-null     object 
 1   유형2        71 non-null     object 
 2   보험명        71 non-null     object 
 3   상세 설명      71 non-null     object 
 4   보장금액       71 non-null     int64  
 5   월 보험료      71 non-null     int64  
 6   기본 보험료     71 non-null     int64  
 7   할증 보험료     17 non-null     float64
 8   납입기간/보험기간  71 non-null     object 
 9   그룹         20 non-null     object 
dtypes: float64(1), int64(3), object(6)
memory usage: 5.7+ KB


In [4]:
data.shape

(71, 10)

데이터는 71개의 행과 10개의 column을 가지고 있다.   
할증 보험료와 그룹에서는 null 값이 존재 하는데, 이는 할증 보험료가 없는 항목과 그룹으로 지정된 몇몇의 보험이 존재하기 때문이다.

## 변수 분리하기

### 납입기간/보장기간 -> 납입기간, 갱신여부, 보장기간
납입기간/보장기간 변수는 __납입기간, 갱신여부, 보장기간__ 이란 3가지 변수를 포함하고 있다.   
이 변수를 각각의 변수로 분리하자.

In [5]:
# ['보험기간', '보장기간']으로 나누기

def split_data(x):
    """납입기간과 보험기간을 분리"""
    if ' ' in x:
        period = x.split(' ')[0]
        guarantee = x.split(' ')[1]
    elif '\n' in x:
        period = x.split('\n')[0]
        guarantee = x.split('\n')[1]

    return [period, guarantee]

data[['납입기간', '보장기간']] = data['납입기간/보험기간'].apply(lambda x: pd.Series(split_data(x)))
data.drop('납입기간/보험기간', inplace=True, axis=1)

In [6]:
# ['보험기간', '갱신']으로 나누기
data[['납입기간(년)', '갱신여부']] = data['납입기간'].str.split('년', expand=True)
data.drop('납입기간', inplace=True, axis=1)
data.sample(10)

,유형1,유형2,보험명,상세 설명,보장금액,월 보험료,기본 보험료,할증 보험료,그룹,보장기간,납입기간(년),갱신여부
33,수술,상해,상해 5종 수술비(시술포함),보험기간 중 상해의 치료를 직접적인 목적으로 약관에서 정한 5종 수술 및 시술을 받...,5000000,267,267,NaN,c,100세만기,20,납
28,진단,상해,보험료 납입면제대상Ⅱ,"보장개시일 이후 상해사고로 80%이상 후유장해가 발생하거나 약관에서 정 한 암, 뇌...",100000,168,168,NaN,NaN,100세만기,20,납
21,진단,갑상선,갑상선기능저하증 진단비,보험기간 중 약관에서 정한 「갑상선기능저하증」으로 진단 확정된 경우 가입 금액 지급...,1000000,170,170,NaN,NaN,100세만기,20,납
20,수술,골절,5대골절 수술비,상해사고로 약관에 정한 5대골절로 수술받은 경우 가입금액 지급,500000,80,80,NaN,NaN,100세만기,20,납
69,수술,화상,화상 수술비,상해사고로 심재성 2도 이상에 해당하는 화상으로 진단 확정 후 그 치료를 직접적인 ...,300000,11,11,NaN,NaN,100세만기,20,납
63,수술,질병,질병 6종 수술비(시술포함),보험기간 중 질병의 치료를 직접적인 목적으로 약관에서 정한 6종 수술 및 시술을 받...,6000000,5056,3410,1646.0,e,100세만기,20,납
13,수술,질병,111대질병 수술비(22대주요질병),보험기간 중 약관에 정한 22대 주요질병으로 진단 확정되고 그 치료를 직접 적인 목...,1000000,704,482,222.0,NaN,100세만기,20,납
12,수술,질병,111대질병 수술비(19대생활질병),보험기간 중 약관에 정한 19대 생활질병으로 진단 확정되고 그 치료를 직접 적인 목...,100000,22,15,7.0,NaN,100세만기,20,납
18,수술,질병,4대특정질병 치료·수술비(3대특정질병),"보험기간 중 약관에 정한 체외충격파쇄석술(급여) 치료, 제왕절개만출술(급 여) 수술...",200000,124,124,NaN,f,100세만기,20,납
0,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원 제외),"상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7305,7305,NaN,NaN,100세만기,20,납


## 파생 변수 만들기

### 총 보험료, 보장 대비 보험료, scaled_보대보

납입 기간 동안 내는 __"총 보험료"__와 총 보험금과 보장에 대한 비율인 __"보장 대비 보험료"__ 변수를 만들자.

두 변수는 아래의 간단한 식을 사용해 구한다.

+ $총 보험료 = 월 보험료 \times 납입기간 \times 12$  
+ $보장\ 대비\ 보험료 = \frac{총 보험료}{보장 금액}$  
+ $sacled\_보대보 = MinMaxscaler(보장\ 대비\ 보험료)$

In [7]:
# 총 보험료 구하기
data['총 보험료'] = data['월 보험료'] * data['납입기간(년)'].astype('int') * 12
data.head()

,유형1,유형2,보험명,상세 설명,보장금액,월 보험료,기본 보험료,할증 보험료,그룹,보장기간,납입기간(년),갱신여부,총 보험료
0,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원 제외),"상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7305,7305,NaN,NaN,100세만기,20,납,1753200
1,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원),"상해사고로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한 간병인을 ...",30000,393,393,NaN,NaN,100세만기,20,납,94320
2,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원 제외),보험기간 중 질병으로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 ...,150000,25297,17265,8032.0,NaN,100세만기,20,납,6071280
3,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원),"보험기간 중 질병으로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한...",10000,3438,2374,1064.0,NaN,100세만기,20,납,825120
4,입원,간병인2,"(체증형,20년후2배)상해 입원 간병인 사용일당(1 일이상)(요양병원 제외)","상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7695,7695,NaN,NaN,100세만기,20,납,1846800


In [8]:
# 납 대비 보장율 구하기
data['보장 대비 보험료'] = (data['총 보험료'] / data['보장금액']).round(3)
data.head()

,유형1,유형2,보험명,상세 설명,보장금액,월 보험료,기본 보험료,할증 보험료,그룹,보장기간,납입기간(년),갱신여부,총 보험료,보장 대비 보험료
0,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원 제외),"상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7305,7305,NaN,NaN,100세만기,20,납,1753200,11.688
1,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원),"상해사고로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한 간병인을 ...",30000,393,393,NaN,NaN,100세만기,20,납,94320,3.144
2,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원 제외),보험기간 중 질병으로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 ...,150000,25297,17265,8032.0,NaN,100세만기,20,납,6071280,40.475
3,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원),"보험기간 중 질병으로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한...",10000,3438,2374,1064.0,NaN,100세만기,20,납,825120,82.512
4,입원,간병인2,"(체증형,20년후2배)상해 입원 간병인 사용일당(1 일이상)(요양병원 제외)","상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7695,7695,NaN,NaN,100세만기,20,납,1846800,12.312


In [9]:
scaler = MinMaxScaler()
data['scaled보대보'] = scaler.fit_transform(data[['보장 대비 보험료']])
data['scaled보대보'] = (data['scaled보대보']* 100).round(3)
data.head()

,유형1,유형2,보험명,상세 설명,보장금액,월 보험료,기본 보험료,할증 보험료,그룹,보장기간,납입기간(년),갱신여부,총 보험료,보장 대비 보험료,scaled보대보
0,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원 제외),"상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7305,7305,NaN,NaN,100세만기,20,납,1753200,11.688,14.132
1,입원,간병인1,(체증형)상해 입원 간병인 사용일당(1일이상)(요양병원),"상해사고로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한 간병인을 ...",30000,393,393,NaN,NaN,100세만기,20,납,94320,3.144,3.802
2,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원 제외),보험기간 중 질병으로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 ...,150000,25297,17265,8032.0,NaN,100세만기,20,납,6071280,40.475,48.940
3,입원,간병인1,(체증형)질병 입원 간병인 사용일당(1일이상)(요양병원),"보험기간 중 질병으로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한...",10000,3438,2374,1064.0,NaN,100세만기,20,납,825120,82.512,99.768
4,입원,간병인2,"(체증형,20년후2배)상해 입원 간병인 사용일당(1 일이상)(요양병원 제외)","상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, ...",150000,7695,7695,NaN,NaN,100세만기,20,납,1846800,12.312,14.887


In [10]:
data[data['scaled보대보'] < 1].sort_values('scaled보대보', ascending=False).tail(5)

,유형1,유형2,보험명,상세 설명,보장금액,월 보험료,기본 보험료,할증 보험료,그룹,보장기간,납입기간(년),갱신여부,총 보험료,보장 대비 보험료,scaled보대보
36,수술,상해,상해 8종 수술비(시술포함),보험기간 중 상해의 치료를 직접적인 목적으로 약관에서 정한 8종 수술 및 시술을 받...,25000000,266,266,NaN,c,100세만기,20,납,63840,0.003,0.004
8,생활,일상배상,[갱신형] 가족 일상생활중 배상책임Ⅱ,"본인 및 약관에 정한 가족의 일상생활 및 보험증권에 기재된 주택의 소유, 사용 또는...",100000000,1154,1154,NaN,NaN,100세만기,20,갱신,276960,0.003,0.004
10,수술,로봇수술,[갱신형] 특정암 다빈치로봇 수술비,"보장개시일 이후 특정암(갑상선암, 전립선암)으로 진단 확정되고 그 특정암 의 직접적...",10000000,200,200,NaN,NaN,100세만기,10,갱신,24000,0.002,0.002
57,진단,화상,중대 화상·부식 진단비,상해사고로 신체 표면적으로 최소 20%이상의 3도 화상 또는 부식(화학약 품 등에 ...,10000000,51,51,NaN,NaN,100세만기,20,납,12240,0.001,0.001
11,생활,일상배상,[공제][갱신형] 가족 일상생활중 배상책임Ⅱ,"※ 1사고당 자기부담금 : 대인 없음, 대물누수사고 50만원, 대물누수사고외 20만원",500000,0,0,NaN,NaN,100세만기,20,갱신,0,0.000,0.000


## 상세 설명에서 데이터 추출
상세 설명 변수는 각 보험에 대한 복잡한 설명을 포함하고 있어서 분석에 적합하지 않게 보인다. 하지만 동시에 보상의 범위라던지, 얼마나 보상을 많이 해줄 수 있는 지 등 중요한 정보도 가지고 있다.

해당 변수는 텍스트가 길고 길이가 rows가 적지 않기 때문에 일일이 읽어 보며 요약하고 추출하는 것은 비효율적일지도 모른다.

ChatGPT가 개발되기 전이라면 당연히 수작업으로 했겠지만, 이제는 GPT의 힘을 빌려보자.

이번 세션에선 ChatGPT에 적절한 프롬프트를 얹어 __"상세 설명"__ column에서 중요한 필요한 파생 변수를 뽑아 내는 작업을 할 것이다.

In [11]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [12]:
import ast # python code 형태의 문자열을 실제 python code로 변환하는 모듈
import traceback
from openai import OpenAI

In [13]:
# API KEY
API_KEY = ""


ChatGPT api에서 적절한 답변을 얻기 위해서는 __적절한 요구사항__이 필요하다.
이를 prompt engineering이라고 하는데, 이번 데이터 추출에 필요한 prompt를 다음 코드처럼 작성하였다.

In [14]:
client = OpenAI(api_key=API_KEY)

def request_openai(document):
    '''content를 담고 있는 list를 받아 chat gpt api로부터 답변을 받아냄.'''

    try:
        response = client.chat.completions.create(
            model='gpt-3.5-turbo',
            messages=[
                {'role': 'system', 'content': '이제부터 너는 보험문서를 검토하는 문서검토 전문가야.'},
                {
                    'role': 'user',
                    'content': '다음 내용은 문서 검토를 하는 예시야. 잘 보고 비슷한 패턴으로 자료에서 필요한 내용을 찾아줘.'
                },
                {
                    'role': 'user',
                    'content': """
                    index: 0
                    <문서 내용>
                    상해사고로 병·의원(요양병원 제외) 등에 1일이상 계속 입원하여 치료를 받 으며, 약관에 정한 간병인을 사용한 경우 1일당 간병인 사용금액을 기준으 로 차등지급(180일을 한도로 간병인 사용 1일당 일당지급)\n※ (1일당 간병인 사용금액 기준) 7만원 미만인 경우 : 가입금액의 50%를 매5년마다 10%씩 정액 할증한 금액 지급, 7만원 이상인 경우 : 가입금액의 100%를 매5년마다 10%씩 정액 할증한 금액 지급\n※ 1일당 간병인 사용금액은 연속적인 간병인 사용일마다 총 사용금액을 총 사용일수로 나눈 금액으로 판단함
                    === 검토 결과 ===
                    보험 발생(질병): 간병인(요양병원 제외)
                    보장 횟수: 1일 일당
                    내용 요약: 상해로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 7만원 미만 50%, 7만원 이상 100% 할증 지급, 최대 180일.

                    index: 1
                    <문서 내용>
                    상해사고로 요양병원에 1일이상 계속 입원하여 치료를 받으며, 약관에 정한 간병인을 사용한 경우 가입후 20년 미만 시 가입금액의 100%를 지급, 가입후 20년 이상 시 가입금액의 200%를 지급(180일을 한도로 간병인 사용 1일당 일당지급)
                    === 검토 결과 ===
                    보험 발생(질병): 간병인(요양병원)
                    보장 횟수: 1일 일당
                    내용 요약: 상해로 1일 이상 요양병원 입원시 간병인 사용에 따라 일당 지급. 20년 미만 50%, 20년 이상 200% 할증 지급, 최대 180일.


                    index: 2
                    <문서 내용>
                    상해사고로 약관에 정한 5대골절로 수술받은 경우 가입금액 지급
                    === 검토 결과 ===
                    보험 발생(질병): 5대골절 수술
                    보장 횟수: 수술시
                    내용 요약: 5대 골절 수술 시 가입금액 지급

                    index: 3
                    <문서 내용>
                    보장개시일 이후 특정암(갑상선암, 전립선암)으로 진단 확정되고 그 특정암 의 직접적인 치료를 목적으로 다빈치 로봇수술을 받은 경우 가입금액 지급( 최초 1회한)\n※ 특정암의 보장개시일은 최초 계약일 또는 부활(효력회복)일부터 90일이 지난 날의 다음날임\n※ 최초 보험가입후 180일 미만에 보험금 지급사유가 발생한 경우 가입금 액의 75% 감액 지급(25%지급), 1년 미만에 보험금 지급사유가 발생한 경우 50% 감액 지급함
                    === 검토 결과 ===
                    보험 발생(질병): 갑상선암, 전립선암
                    보장 횟수: 최초 1회
                    내용 요약: 특정암 진단 후 다빈치 수술 시 가입금액 지급, 보장개시 90일 후, 초기 감액 있음.

                    index: 4
                    <문서 내용>
                    보험기간 중 약관에 정한 뇌질환, 심질환, 간·췌장질환 및 폐질환으로 진단 확정되고 그 치료를 직접적인 목적으로 내시경수술, 카테터수술, 신의료수 술 이외의 수술 중 약관에 정한 관혈수술을 받은 경우 가입금액 지급(수술 1 회당)\n※ 보험가입후 1년 미만에 보험금 지급사유가 발생한 경우 50% 감액 지급\n※ 단, 수술 중 비관혈수술과 관혈수술이 동시에 행해진 경우에는 관혈수술 비만 지급
                    === 검토 결과 ===
                    보험 발생(질병): 뇌질환, 심질환, 간·췌장질환 및 폐질환
                    보장 횟수: 수술당 1회
                    내용 요약: 뇌, 심, 간·췌장, 폐질환 관혈수술 시 가입금액 지급, 1년 미만 50% 감액.

                    index: 5
                    <문서 내용>
                    상해사고로 심재성 2도 이상에 해당하는 화상으로 진단 확정 후 그 치료를 직접적인 목적으로 수술을 받은 경우 가입금액 지급
                    === 검토 결과 ===
                    보험 발생(질병): 화상
                    보장 횟수: 수술시
                    내용 요약: 심재성 2도 이상 화상 수술 시 가입금액 지급

                    index:6
                    <문서 내용>
                    ※ 1사고당 자기부담금 : 대인 없음, 대물누수사고 50만원, 대물누수사고외 20만원
                    === 검토 결과 ===
                    보험 발생(질병): 사고
                    보장 횟수: 1사고당
                    내용 요약: 1사고당 자기부담금: 대인 없음, 대물누수 50만원, 기타 대물 20만원
                    """
                },
                {
                    'role': 'user', 'content': """
                    아래는 대답할 답변의 조건이야. 반영해서 대답해줘.

                    1. '보험 발생(질병)', '보장 횟수'에 대한 결과는 예시처럼 최대한 간결하게 만들어줘.
                    2. 결과물을 DataFrame에 추가할 수 있도록 아래처럼 dictionary 형태로 만들어줘.
                    3. ast로 바로 적용할 수 있게 python code 형태로 반환해줘.
                    4. 결과를 변수에 대입하는 코드는 빼줘. 깔끔한 딕셔너리 타입으로 반환해줘.

                    예시
                    '''
                    {
                        'index': [0, 1, 2, 3, 4, 5],
                        '보험 발생(질병)': ['간병인(요양병원 제외)', '간병인(요양병원)', '5대골절 수술', '갑상선암, 전립선암', '간·췌장질환 및 폐질환', '사고'],
                        '보장 횟수': ['1일 일당', '1일 일당','수술받은 경우', '최초 1회', '수술당 1회', '사고 당'],
                        '내용 요약' : [
                            '상해로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 7만원 미만 50%, 7만원 이상 100% 할증 지급, 최대 180일.',
                            '상해로 1일 이상 요양병원 입원시 간병인 사용에 따라 일당 지급. 20년 미만 50%, 20년 이상 200% 할증 지급, 최대 180일.',
                            '5대 골절 수술 시 가입금액 지급',
                            '특정암 진단 후 다빈치 수술 시 가입금액 지급, 보장개시 90일 후, 초기 감액 있음.',
                            '심재성 2도 이상 화상 수술 시 가입금액 지급',
                            '1사고당 자기부담금: 대인 없음, 대물누수 50만원, 기타 대물 20만원',
                        ],
                    }
                    '''

                    이제부터 아래는 검토해야 할 문서들을 알려줄게.
                    """
                },
                {'role': 'user', 'content': document},
            ]
        )

        content = response.choices[0].message.content

    except:
        print(traceback.print_exc())

    return content

ChatGPT API에서 답변을 구할 때 좀 더 효율적으로 자원을 쓸 수 있는 방법으로 알려진 것은 "batch processing" 또는 "batch request"라고 알려진 방법이다.

이는 하나의 request에 여러 개의 prompt를 태우는 것으로 한 번에 여러 개의 답변을 받을 수 있다.

request를 batch로 처리하기 위해 prompt를 하나로 묶자.


In [15]:
# batch size
BATCH_SIZE = 6

In [16]:
data_list = data['상세 설명'].to_list()

contents = []
doc_index = 0

# BATCH_SIZE 만큼 "상세 설명"을 문서 내용에 포맷팅하여 추가
for i in range(0, len(data_list), BATCH_SIZE):

    content = ''

    chunks = data_list[i:i+BATCH_SIZE]
    for chunk in chunks:
        content += f"""
        index: {doc_index}
        <문서 내용>
        {chunk}

        """
        doc_index += 1

    contents.append(content)


In [17]:
# contents의 샘플을 살펴보자.
print(contents[4])


        index: 24
        <문서 내용>
        상해사고로 약관에 정한 골절(치아파절(깨짐, 부러짐) 제외)로 진단 확정된 경우 가입금액 지급

        
        index: 25
        <문서 내용>
        보장개시일 이후 기타피부암 또는 갑상선암으로 진단 확정되고 「기타피부암 및 갑상선암 보험금 지급기간」이내에 「기타피부암 및 갑상선암 특정치료」를 받은 경우 가입금액을 지급(연간 1회한)
※ 기타피부암 또는 갑상선암의 보장개시일은 최초 계약일 또는 부활(효력 회복)일부터 90일이 지난 날의 다음날임
※ 기타피부암 및 갑상선암 보험금 지급기간이란 기타피부암 또는 갑상선암 최초 진단확정일을 포함하여 5년까지의 기간으로 보험금 지급대상이 되는 기간을 말함
※ 기타피부암 및 갑상선암 특정치료는 수술, 항암방사선치료, 항암약물치 료 및 말기암 환자에 대한 호스피스완화의료 치료를 말함

        
        index: 26
        <문서 내용>
        보장개시일 이후 기타피부암 또는 갑상선암으로 진단 확정된 경우 가입금액 지급(최초 1회한)
※ 기타피부암 또는 갑상선암의 보장개시일은 최초 계약일 또는 부활(효력 회복)일부터 90일이 지난 날의 다음날임

        
        index: 27
        <문서 내용>
        보험기간 중 상해 또는 질병으로 약관에 정한 깁스치료를 받은 경우 가입금 액 지급
※ 부목치료(Splint Cast)는 제외

        
        index: 28
        <문서 내용>
        보장개시일 이후 상해사고로 80%이상 후유장해가 발생하거나 약관에서 정 한 암, 뇌졸중, 급성심근경색증, 중대 화상·부식으로 진단 확정 또는 뇌·내장 손상 수술로 보험료 납입면제사유가 발생한 경우 가입금액 지급(최초 1회한
)
※ 암으로 인한 납입면제의 보장개시일은 최초 계약일 또는 부활(효력회복) 일부터 90일이 지난 날의 다음날이며

In [18]:
# 이제 각 문서를 ChatGPT api에 태워 결과를 받아 보자.

In [19]:
gpt_responses = []

for content in contents:
    response = request_openai(content)
    gpt_responses.append(response)

print("Done.")

Done.


In [20]:
# 결과물에 이상이 없는지 살펴보자
for response in gpt_responses:
    print(response)

```
{
    'index': [0, 1, 2, 3, 4, 5],
    '보험 발생(질병)': ['상해사고 간병인(요양병원 제외)', '상해사고 간병인', '질병 간병인(요양병원 제외)', '질병 간병인', '상해사고 간병인(20년 이내)', '상해사고 간병인(20년 이상)'],
    '보장 횟수': ['1일 일당', '1일 일당','1일 일당', '1일 일당', '1일 일당', '1일 일당'],
    '내용 요약': [
        '상해로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 7만원 미만 50%, 7만원 이상 100% 할증 지급, 최대 180일.',
        '상해로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 10%씩 정액 할증, 최대 180일.',
        '질병으로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 7만원 미만 50%, 7만원 이상 100% 할증 지급, 최대 180일.',
        '질병으로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 10%씩 정액 할증, 최대 180일.',
        '상해로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 20년 미만 100%, 20년 이상 200% 할증 지급, 최대 180일.',
        '상해로 1일 이상 입원 시 간병인 사용에 따라 일당 지급. 20년 미만 100%, 20년 이상 200% 할증 지급, 최대 180일.'
    ]
}
```
```
{
    'index': [6, 7, 8, 9, 10, 11],
    '보험 발생(질병)': ['간병인(요양병원 제외)', '간병인(요양병원)', '본인 및 가족의 사고', '암(특정암 제외)', '특정암(갑상선암, 전립선암)', '사고'],
    '보장 횟수': ['가입후 20년 미만 50%, 가입후 20년 이상 100%, 180일 한도', '가입후 20년 미만 100%, 가입후 20년 이상 200%, 180일 한도', '1회한', '최초 1회', '최초 1회', '1사고당'

다음 과정은 ChatGPT api로 받은 결과물을 손질하고 __ast__ 모듈을 사용하여 python dictionary로 만드는 작업이다.  

단, 이때 종종 GPT의 결과물이 python code 형식과 다른 경우가 발생할 수 있다.   
이런 경우 해당 데이터를 찾아서 수정하거나, 다시 한번 api 호출을 통해 데이터를 받을 수 있다.

In [21]:
clean_answers = []


for response in gpt_responses:
    try:
        # 각 대답에 의미 없는 문자열을 제거하여 dictionary 형태로 정리한다.
        response = response.replace('`', '')
        response = response.replace('\n', '')
        response = response.replace('=', '')
        response = response.replace('python', '')
        response = response.replace("'''", '')
        response = "'''" + response + "'''"

        # dictionary 타입으로 변환
        response = ast.literal_eval(response)
        if isinstance(response, str):
            response = ast.literal_eval(response)

        clean_answers.append(response)

    except (SyntaxError, ValueError) as e:
        print(traceback.print_exc())
        print(response)
        break

In [22]:
for i, answer in enumerate(clean_answers):
    print(i, type(answer))

0 <class 'dict'>
1 <class 'dict'>
2 <class 'dict'>
3 <class 'dict'>
4 <class 'dict'>
5 <class 'dict'>
6 <class 'dict'>
7 <class 'dict'>
8 <class 'dict'>
9 <class 'dict'>
10 <class 'dict'>
11 <class 'dict'>


모든 데이터가 dictionary 형태로 반환된 것을 확인하였다면, 이제는 dataframe에 합치는 작업을 수행하자.

In [23]:
result_df=pd.DataFrame(columns=['보험 발생(질병)', '보장 횟수', '내용 요약'])

for answer in clean_answers:
    temp_df = pd.DataFrame(answer)
    result_df = pd.concat([result_df, temp_df], join='inner').reset_index(drop=True)

result_df.rename(columns={'보험 발생(질병)':'보장 발생'}, inplace=True)

In [24]:
# 병합할 DataFrame 살펴보기
display(result_df['보장 발생'].value_counts())
print("="*100)
display(result_df['보장 횟수'].value_counts())

보장 발생
암, 기타피부암 또는 갑상선암                    9
상해                                  6
화상                                  3
뇌질환, 심질환, 간·췌장질환 및 폐질환              2
기타피부암, 갑상선암                         2
8종 수술 및 시술                          2
간호·간병통합서비스                          2
질병                                  2
골절 수술                               1
사망                                  1
후유장해                                1
아킬레스힘줄손상 수술                         1
암                                   1
암(기타피부암 및 갑상선암 제외)                  1
상해사고 간병인(요양병원 제외)                   1
기타피부암, 갑상선암, 대장점막내암, 제자리암, 경계성종양    1
상해의 수술 및 시술                         1
인공관절치환 수술                           1
암, 기타피부암, 갑상선암                      1
3종 수술 및 시술                          1
4종 수술 및 시술                          1
5종 수술 및 시술                          1
6종 수술 및 시술                          1
7종 수술 및 시술                          1
급성심근경색증                             1
아나필락시스(anaphylaxis)                 1
깁스치료  

보장 횟수
수술시술코드당 연간1회한                                15
연간 1회한                                       12
1일 일당                                         7
최초 1회                                         6
수술당 1회                                        6
최초 1회한                                        5
연간 1회                                         3
수술받은 경우                                       3
1회                                            1
수술시                                           1
차회 이후 이 계약의 보험료 납입시마다                         1
수술을 받은 경우                                     1
한도로                                           1
1일당                                           1
치료 시                                          1
1종 수술 및 시술코드당 연간1회한                           1
가입후 20년 미만 50%, 가입후 20년 이상 100%, 180일 한도      1
수술로                                           1
1사고당                                          1
1회한                                           1
가입후 20년 미만 100%, 가입후 20년 이상 200%, 

In [26]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   보장 발생   71 non-null     object
 1   보장 횟수   71 non-null     object
 2   내용 요약   71 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB


In [27]:
new_data = data.join(result_df)

In [28]:
new_data.tail(10)

,유형1,유형2,보험명,상세 설명,보장금액,월 보험료,기본 보험료,할증 보험료,그룹,보장기간,납입기간(년),갱신여부,총 보험료,보장 대비 보험료,scaled보대보,보장 발생,보장 횟수,내용 요약
61,수술,질병,질병 4종 수술비(시술포함),보험기간 중 질병의 치료를 직접적인 목적으로 약관에서 정한 4종 수술 및 시술을 받...,450000,3646,2453,1193.0,d,100세만기,20,납,875040,1.945,2.352,4종 수술 및 시술,수술시술코드당 연간1회한,4종 수술 및 시술 시 가입금액 지급. 1회의 입원 또는 통원 당 1회 보장
62,수술,질병,질병 5종 수술비(시술포함),보험기간 중 질병의 치료를 직접적인 목적으로 약관에서 정한 5종 수술 및 시술을 받...,3000000,5058,3411,1647.0,e,100세만기,20,납,1213920,0.405,0.490,5종 수술 및 시술,수술시술코드당 연간1회한,5종 수술 및 시술 시 가입금액 지급. 1회의 입원 또는 통원 당 1회 보장
63,수술,질병,질병 6종 수술비(시술포함),보험기간 중 질병의 치료를 직접적인 목적으로 약관에서 정한 6종 수술 및 시술을 받...,6000000,5056,3410,1646.0,e,100세만기,20,납,1213440,0.202,0.244,6종 수술 및 시술,수술시술코드당 연간1회한,6종 수술 및 시술 시 가입금액 지급. 1회의 입원 또는 통원 당 1회 보장
64,수술,질병,질병 7종 수술비(시술포함),보험기간 중 질병의 치료를 직접적인 목적으로 약관에서 정한 7종 수술 및 시술을 받...,9000000,5056,3410,1646.0,e,100세만기,20,납,1213440,0.135,0.163,7종 수술 및 시술,수술시술코드당 연간1회한,7종 수술 및 시술 시 가입금액 지급. 1회의 입원 또는 통원 당 1회 보장
65,수술,질병,질병 8종 수술비(시술포함),보험기간 중 질병의 치료를 직접적인 목적으로 약관에서 정한 8종 수술 및 시술을 받...,15000000,5056,3410,1646.0,e,100세만기,20,납,1213440,0.081,0.098,8종 수술 및 시술,수술시술코드당 연간1회한,8종 수술 및 시술 시 가입금액 지급. 1회의 입원 또는 통원 당 1회 보장
66,입원,간호간병,질병 입원 간호·간병통합서비스 사용일당(1일이상),보험기간 중 질병으로 병원(요양병원 제외)에 1일이상 계속 입원하여 치료 를 받으며...,70000,3425,2331,1094.0,NaN,100세만기,20,납,822000,11.743,14.199,간호·간병통합서비스,1일 일당,"질병으로 1일 이상 입원 시 간호·간병통합서비스 사용에 따라 일당 지급, 최대 180일."
67,수술,심근경색,혈전용해치료비(급성심근경색증),"보험기간 중 약관에서 정한 급성심근경색증으로 진단 확정되고, 그 치료를 직접적인 목...",10000000,445,445,NaN,a,100세만기,20,납,106800,0.011,0.013,급성심근경색증,최초 1회,급성심근경색증 진단 후 혈전용해 치료시 최초 1회 지급
68,수술,심근경색,혈전용해치료비(뇌경색증),"보험기간 중 약관에서 정한 뇌경색증으로 진단 확정되고, 그 치료를 직접적 인 목적으...",10000000,445,445,NaN,a,100세만기,20,납,106800,0.011,0.013,뇌경색증,최초 1회,뇌경색증 진단 후 혈전용해 치료시 최초 1회 지급
69,수술,화상,화상 수술비,상해사고로 심재성 2도 이상에 해당하는 화상으로 진단 확정 후 그 치료를 직접적인 ...,300000,11,11,NaN,NaN,100세만기,20,납,2640,0.009,0.011,화상,수술시,심재성 2도 이상 화상 수술 시 가입금액 지급
70,진단,화상,화상 진단비,상해사고로 심재성 2도 이상에 해당하는 화상으로 진단 확정된 경우 가입금 액 지급,100000,76,76,NaN,NaN,100세만기,20,납,18240,0.182,0.220,화상,없음,심재성 2도 이상 화상 진단시 가입금액 지급


## 저장
전처리가 모두 끝났다. 이제 새로운 데이터로 저장하자.  
해당 데이터는 ',(콤마)'가 많이 등장하므로, csv대신 엑셀로 저장을 수행한다.


그리고 추가적인 수정이 필요한 부분이 있다면 엑셀이나 추가적인 노트북 작업으로 수정을 수행한다.

In [30]:


from google.colab import files

def download_excel(df, filename):
  """
  Downloads a Pandas DataFrame as an Excel file.

  Args:
    df: The Pandas DataFrame to download.
    filename: The name of the file to save.
  """

  # Create a temporary file to store the Excel data.
  temp_file = "/tmp/{}".format(filename)

  # Save the DataFrame to the temporary file.
  df.to_excel(temp_file, index=False)

  # Download the temporary file.
  files.download(temp_file)

# Download the DataFrame as an Excel file.
download_excel(new_data, "preprocessed_data_txt-sum.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>